In [ ]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from global_variables import *

In [ ]:
import pandas as pd
import numpy as np
import folium.features

from python_scripts.miscellaneaous.data_processing import extract_data
from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.ihm.maps.mapUtils import *
from python_scripts.city.city_utils import *

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df = extract_data(df, provider=PROVIDER, min_info=True)

In [ ]:
df_countryside = pd.read_csv("../../database/data_countryside.csv", sep=";", decimal=',')

In [ ]:
df_pop = pd.read_csv("../../database/data_population.csv", sep=";", decimal=',')

In [ ]:
# for com in df['nom_com'].unique():
#     df.loc[df['nom_com']==com, 'countryside'] = (1 if (df_countryside.loc[df_countryside['COM_NOM']==com, 'UU_ID']=='SO').all() else 0)

In [ ]:
for com in df['nom_com'].unique():
    df.loc[df['nom_com']==com, 'countryside'] = (1 if (df_pop.loc[df_pop['Commune']==com, 'PMUN']<7000).all() else 0)

In [ ]:
df.head()

map = folium.Map(location=np.mean(df[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")
points = folium.FeatureGroup(f"Points ({len(G)})").add_to(map)

for bs_id, latitude, longitude in df[['latitude', 'longitude']].itertuples():
    color = 'gray' if (df['countryside'][bs_id]==1) else 'magenta'
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=1, popup=f"{bs_id}"))

folium.LayerControl().add_to(map)

map.save("../../out/maps/countryside_v2.html")

Comparison with other methods

In [ ]:
def mean_distance_to_city_level(v):
    if (v>4):
        return 4
    if (v>2):
        return 3
    if (v>1):
        return 2
    return 1

In [ ]:
labels = df['countryside'].apply(lambda v : -1 if(v==1) else 0)

mean_distances = mean_distance_to_NN(df[['x', 'y']], n_neighbours=3)
labels_3NN_detailed = mean_distances.apply(mean_distance_to_city_level)
labels_3NN = labels_3NN_detailed.apply(lambda v : 0 if(v<=2) else -1)

In [ ]:
labels_dbs_detailed = city_detection(df[['x', 'y']])['labels']
labels_dbs = labels_dbs_detailed.apply(lambda v : -1 if(v==-1) else 0)
labels_hdb_detailed = city_detection_enhanced(df[['x', 'y']])['labels']
labels_hdb = labels_hdb_detailed.apply(lambda v : -1 if(v==-1) else 0)

In [ ]:
print(np.sum(labels==labels_3NN)/len(labels))
print(np.sum(labels==labels_dbs)/len(labels))
print(np.sum(labels==labels_hdb)/len(labels))

In [ ]:
def labelsToColor(l1, l2): 
    if ((l1==-1) and (l2==-1)):
        return 'green'
    if((l1==-1) or (l2==-1)):
        return 'blue'
    return 'red'

In [ ]:
colors = pd.Series(data=[labelsToColor(l, l3NN) for l, l3NN in zip(labels, labels_3NN)], index=df.index)
map1 = folium.Map(location=np.mean(df[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df[['latitude', 'longitude']].itertuples():
    com = df['nom_com'][ind]
    popup = f"{com} [{df_pop.loc[df_pop['Commune']==com,'PMUN'].values[0]} hab] (label, 3-NN) :\n({labels[ind]!=-1},{labels_3NN[ind]!=-1})"
    map1.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map1.save("../../out/maps/city-detection-comparison_3NN_v2.html")

In [ ]:
colors = pd.Series(data=[labelsToColor(l, ldbs) for l, ldbs in zip(labels, labels_dbs)], index=df.index)
map2 = folium.Map(location=np.mean(df[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df[['latitude', 'longitude']].itertuples():
    popup = f'ville (label, DBScan) :\n({labels[ind]!=-1},{labels_dbs[ind]!=-1})'
    map2.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map2.save("../../out/maps/city-detection-comparison_DBScan_v2.html")

In [ ]:
colors = pd.Series(data=[labelsToColor(l, lhdb) for l, lhdb in zip(labels, labels_hdb)], index=df.index)
map3 = folium.Map(location=np.mean(df[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")

for ind, latitude, longitude in df[['latitude', 'longitude']].itertuples():
    popup = f'ville (label, HDBScan) :\n({labels[ind]!=-1},{labels_hdb[ind]!=-1})'
    map3.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, color = colors[ind], popup=popup))

map3.save("../../out/maps/city-detection-comparison_HDBScan_v2.html")